In [1]:
#모듈 로딩
#### 모듈 및 라이브러리 로딩
from bs4 import BeautifulSoup #뷰티풀숲 로딩
from selenium import webdriver #셀레니움 드라이버 로딩
from selenium.webdriver.common.keys import Keys #키에 관련된 모듈 가져오기
import time #시간 체크 및 관리를 위한 모듈 로딩

import sys #파일로 저장하기 위한 모듈
import os #폴더를 다루기 위한 모듈
import pandas as pd #데이터 프레임으로 만들기 위한 모듈
import xlwt #엑셀로 저장을 위해 
import openpyxl#엑셀 파일을 다루기 위한 모듈
import csv #csv 모듈

import re #정규식 모듈 임포트, 파일명 잘못 생성시 예외처리를 위한 모듈
from datetime import datetime#날짜 계산을 위한 타임모듈


#사용자 입력 받는 부분과, 최초 실행시간 코드 지정
print('''
=========================================================================
==================== 게임 카페 게시판 크롤러 ver 1.0 ====================

           본 데이터는 최신게임과 커뮤니티의 규모를 고려하여
            네이버 카페 게임 섹션의 급상승 Top100 리스트와
       실시간 Top100 리스트를 비교, 두 리스트에 공통으로 포함된 
                   카페의 게시판 데이터를 추출합니다.
             
=========================================================================''')


#입력값이 비었이면 False, 아니면 True-------------------------------------------------------
def isNotBlank (myString):
    return bool(myString and myString.strip())

#수집할 기간 입력 --------------------------------------------------------------------------
while True:
    try:
        limit_day =int(input('데이터를 추출할 기간을 입력해 주세요: '))  
    except ValueError:
        print('양의 정수 중에서 입력해 주세요.')
    else:
        if limit_day==0:
            limit_day = 1
            print("입력값이 없다면 최소단위인 1일간의 수집된 데이터를 추출시작합니다.")
        else:
            limit_day = abs(limit_day)
            print('%s일간의 수집된 데이터를 추출시작합니다.'%limit_day)
        break
        

#크롤링 시작 시간을 위한 타임스탬프를 찍습니다.
s_time = time.time()


#브라우저 오픈 및 정렬 기준 변경 -------------------------------------------------------------
def open_br():
    global driver
    global path

    path = 'chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get("https://section.cafe.naver.com/ca-fe/home/themes")#네이버 카페 주제별 링크
    time.sleep(1)
    
    #정렬기준 멤버순 변경
    driver.find_element_by_xpath ( '// button [normalize-space () = "급상승 순"]').click()
    time.sleep(0.5)
    driver.find_element_by_xpath ( '// button [normalize-space () = "멤버 순"]').click()    
    time.sleep(0.5)
        
open_br()

#주제별 카페에서 급상승 Top100의 카페 이름과 URL을 수집하는 클래스-----------------------------
class CrawlingTop100:
    def __init__(self, dic):
        self.dic = dic
        #dic = dict()
        self.active_code_end = True
        
    def crawling_page(self): 
        self.no = 1
        for x in range(1, 7):
            time.sleep(1)
            self.html = driver.page_source#페이지 전체를 가져옴
            self.soup = BeautifulSoup(self.html, 'html.parser')
            self.content_list = self.soup.find('ul', 'common_list').find_all('li')

            for i in self.content_list:
                #print('번호:', self.no)
                self.title = i.find('div', 'list_info').find('div', 'name_area').get_text()
                #print(self.title.strip())
                self.url = i.find('a', 'list_link')['href']
                #print(self.url)      

                self.dic[self.no] = [self.title, self.url]
                self.no+=1
            time.sleep(1)

            self.page_num= str(x+1)
            if (x > 0) and (x < 6) :
                driver.find_element_by_link_text(self.page_num).click()
            x += 1
            
            if self.no == 100:
                break     
        return self.dic
        
        return self.active_code_end
    
    
    
#Top100들에서 타이틀과 URL 수집------------------------------------------------------
dic_a = {}
dic_b = {}
crwling_top100_a = CrawlingTop100(dic_a)
crwling_top100_b = CrawlingTop100(dic_b)

crwling_top100_a.crawling_page()

#급상승 끝나면 랭킹으로 이동
if crwling_top100_a.active_code_end == True:
    driver.find_element_by_link_text('랭킹 Top100').click()
    time.sleep(1)
    crwling_top100_b.crawling_page()
    
    
    
#같은 값의 주소리스트와 갯수 수집--------------------------------------------------
list_a = []
list_b = []
for i in dic_a:
    title = dic_a[i][1]
    list_a.append(title)
for i in dic_b:
    title = dic_b[i][1]
    list_b.append(title)

list_ab = [val for val in list_a if val in list_b]


print('급상승 Top100과 랭킹 Top100에 모두 포함된 카페의 수는 %s 건 입니다.'%str(len(list_ab)))
print('='*73)


#각 카페에서 데이터 수집하는 클래스(no)---------------------------------------------
class CrawlingCafe:
    def __init__(self, url, limit_day):
        self.url = url
        self.limit_day = limit_day
        
    def crawling_cafe(self):
        driver.get(self.url)
        
        self.html = driver.page_source
        self.soup = BeautifulSoup(self.html, 'html.parser')
         
        self.g_name = []
        self.gc_name = []
        self.gc_url = []

        self.gc_total_member = []

        self.gc_total_board_count = []

        self.gc_board_num = []
        self.gc_board_category = []
        self.gc_board_date = []
        self.gc_board_title = []
        
        self.rg_name = self.soup.find('title').get_text()
        self.re_rg_name = re.sub('[\\\/:*?"<>|\]\[-]','',self.rg_name)
        self.rg_name = re.sub('공식.*\Z|네이버.*\Z|커뮤니티.*\Z|카페.*\Z','',self.re_rg_name)#공식 뒤의 문자를 모두 날림

        self.rgc_name = self.soup.find('title').get_text()
        
        self.rgc_url = driver.current_url#현재 브라우저 URL

        self.rgc_total_member = self.soup.find('li', 'mem-cnt-info').find('a').get_text()
        self.rgc_total_member = re.sub('비공개','', self.rgc_total_member)
        self.rgc_total_member = self.rgc_total_member.replace('\n', '')

        self.rgc_total_board_count = self.soup.find('ul', 'cafe-menu-list').find('li').find('span').get_text()                  
        
       
        driver.find_element_by_link_text('전체글보기').click()

                
                
        time.sleep(0.5)
        
            
        self.today = datetime.today().strftime('%Y-%m-%d')
        self.re_today = re.sub('-', '', self.today)             

        #----------자유게시판 iframe이동----------------
        driver.switch_to.frame("cafe_main")

        self.no = 1
        self.x = 1
        while True:
            time.sleep(0.5)
            self.iframe = driver.page_source
            self.soup = BeautifulSoup(self.iframe, 'html.parser')
            self.board = self.soup.find_all('div', 'article-board m-tcol-c')
            self.board2 = self.board[1].find('tbody').find_all('tr')

            for i in self.board2:              
                try:
                    self.title = i.find('div','board-list').find('div','inner_list').find('a', 'article').get_text()
                except:
                    continue
                else:
                    self.title = self.title.replace("\n","")
                    self.title = self.title.replace("\t","")

                try:
                    self.w_time = i.find('td','td_date').get_text()
                except:
                    continue
                else:
                    self.w_time = self.w_time.replace("\n","")
                    self.w_time = self.w_time.replace("\t","")
                    #기간설정을 위한 w_time 변환
                    self.re_time = re.sub('[:.]','',self.w_time)
                    
                try:
                    self.category = i.find('div','board-name').find('div','inner_name').find('a').get_text()
                except:
                    continue
                else:
                    self.category = self.category.replace("\n","")
                    self.category = self.category.replace("\t","")                                      
                
                #기간 설정 조건 비교 후 크롤링 끝내기 ----------                   
                if self.limit_day == 1:
                    if len(self.re_time) > 4:
                        break
                else:
                    self.crawling_days = int(self.re_today) - self.limit_day
                    if int(self.re_time) == self.crawling_days:
                        break
                #-----------------------------------------------

                self.g_name.append(self.rg_name)
                self.gc_name.append(self.rgc_name)
                self.gc_url.append(self.rgc_url)
                self.gc_total_member.append(self.rgc_total_member)
                self.gc_total_board_count.append(self.rgc_total_board_count)
                
                  
                #print(self.title)
                self.gc_board_title.append(self.title.strip())
                #print(self.w_time)
                self.gc_board_date.append(self.w_time.strip())      
                self.gc_board_category.append(self.category.strip())
                self.no += 1

            self.page_num= str(self.x+1)
            self.list_num = self.page_num

            if (self.x % 10) == 0:
                driver.find_element_by_link_text("다음").click()
            else:
                driver.find_element_by_link_text(self.page_num).click()
            self.x += 1


            #기간 설정 조건 비교 후 크롤링 끝내기 ----------
            #기간설정을 위한 w_time 변환
            self.re_time = re.sub('[:.]','',self.w_time)                
            if self.limit_day == 1:
                if len(self.re_time) > 4:
                    break
            else:
                self.crawling_days = int(self.re_today) - self.limit_day
                if int(self.re_time) == self.crawling_days:
                    break       
        return self.g_name, self.gc_name, self.gc_url, self.gc_total_member, self.gc_total_board_count, self.gc_board_date, self.gc_board_title, self.gc_board_category

    
    
#저장될 파일명 생성, 폴더 변경 ------------------------------------------------------------------------------------
now = time.localtime()
f_name = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

fc_name = f_name+'-'+"gamecafe_data"+'.csv'
fx_name = f_name+'-'+"gamecafe_data"+'.xlsx'
final_f_dir = os.getcwd()+"\\Log\\"   

#폴더 생성 시작
try:
    os.makedirs(final_f_dir)
except FileExistsError:#동일 폴더 생성시 예외처리
    pass

os.chdir(final_f_dir)


#반복 카페 글 크롤링 후 저장 ---------------------------------------------------------------------------------------
writer = pd.ExcelWriter(fx_name, engine='xlsxwriter')
column_width = 20

no = 1

for i in range(len(list_ab)):
    i = CrawlingCafe(list_ab[i], limit_day)
    i.crawling_cafe()
    
    cafe_data = "cafe_data"+str(i)
    cafe_data = pd.DataFrame()
    cafe_data['게임 명'] = i.g_name
    cafe_data['카페 명'] = i.gc_name
    cafe_data['카페 주소'] = i.gc_url
    cafe_data['카페 멤버수'] = i.gc_total_member
    cafe_data['카페 총게시물 수'] = i.gc_total_board_count
    cafe_data['글 번호'] = [i+1 for i in range(len(i.gc_board_title))]
    cafe_data['게시판 이름'] = i.gc_board_category
    cafe_data['글 작성 시간'] = i.gc_board_date
    cafe_data['글 내용'] = i.gc_board_title
    
    cafe_data.to_excel(writer, sheet_name = i.g_name[0], index=False)    
    worksheet = writer.sheets[i.g_name[0]]
    worksheet.set_column(4, 0, column_width)

    print("<---------"+str(no)+"번째 카페 데이터 추출을 마쳤습니다.--------->")
    #print(cafe_data)
    time.sleep(2)
    no+=1
writer.save()

e_time = time.time()
t_time = e_time - s_time #크롤링에 쓰인 시간

#엑셀 파일 csv 시트별로 저장----------------------------------------------------
def convert_csv():
    print('엑셀 파일을 csv로 변환작업 중입니다.')
    time.sleep(4)

    sheetList = []
    #엑셀의 열어 workbook을 얻어옵니다. 
    wb = openpyxl.load_workbook(final_f_dir + fx_name)
    xlsxFile =final_f_dir + fx_name

    for i in wb.sheetnames:
        sheetList.append(i)

    xlsx = pd.ExcelFile(xlsxFile)

    no = 1
    for j in sheetList:
        df = pd.read_excel(xlsx, j)
        df.to_csv(fx_name[:-5]+'_sheet_{}'.format(no)+'.csv', encoding='utf-8-sig', index=False)
        no+=1

convert_csv()

print('='*73)
print('총 소요시간은 %s 초입니다.'%round(t_time, 1))
print('파일 저장 경로는 %s 으로 추출을 마치고 프로그램을 종료합니다.'%final_f_dir)
print('='*73)
driver.close()


==================== 게임 카페 게시판 크롤러 ver 1.0 ====================

           본 데이터는 최신게임과 커뮤니티의 규모를 고려하여
            네이버 카페 게임 섹션의 급상승 Top100 리스트와
       실시간 Top100 리스트를 비교, 두 리스트에 공통으로 포함된 
                   카페의 게시판 데이터를 추출합니다.
             
데이터를 추출할 기간을 입력해 주세요: 1
1일간의 수집된 데이터를 추출시작합니다.
급상승 Top100과 랭킹 Top100에 모두 포함된 카페의 수는 30 건 입니다.
<---------1번째 카페 데이터 추출을 마쳤습니다.--------->
<---------2번째 카페 데이터 추출을 마쳤습니다.--------->
<---------3번째 카페 데이터 추출을 마쳤습니다.--------->
<---------4번째 카페 데이터 추출을 마쳤습니다.--------->
<---------5번째 카페 데이터 추출을 마쳤습니다.--------->
<---------6번째 카페 데이터 추출을 마쳤습니다.--------->
<---------7번째 카페 데이터 추출을 마쳤습니다.--------->
<---------8번째 카페 데이터 추출을 마쳤습니다.--------->
<---------9번째 카페 데이터 추출을 마쳤습니다.--------->
<---------10번째 카페 데이터 추출을 마쳤습니다.--------->
<---------11번째 카페 데이터 추출을 마쳤습니다.--------->
<---------12번째 카페 데이터 추출을 마쳤습니다.--------->
<---------13번째 카페 데이터 추출을 마쳤습니다.--------->
<---------14번째 카페 데이터 추출을 마쳤습니다.--------->
<---------15번째 카페 데이터 추출을 마쳤습니다.--------->
<---------16번째 카페 데이터 추출을 